In [2]:
import pandas as pd
import os
import time
import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
from progress.bar import Bar
import pandas as pd

from utils import loss_funcs, utils as utils
from utils.opt import Options
from utils.h36motion3d import H36motion3D
import utils.model as nnmodel
import utils.data_utils as data_utils
from utils.constants import *
from utils.model import TimeAutoencoder
import matplotlib.pyplot as plt

In [30]:
opt = Options()
opt._initial()

class MyOpt:
    def __init__(self):
        self.output_n = opt.parser.get_default('output_n')
        self.input_n = opt.parser.get_default('input_n')
        self.dct_n = opt.parser.get_default('dct_n')
        self.linear_size = opt.parser.get_default('linear_size')
        self.num_stage = opt.parser.get_default('num_stage')
        self.dropout = opt.parser.get_default('dropout') 
        self.data_dir = opt.parser.get_default('data_dir') 
        self.sample_rate = opt.parser.get_default('sample_rate')
        self.test_batch = opt.parser.get_default('test_batch')
        
        self.job = opt.parser.get_default('job')
        self.test_batch = opt.parser.get_default('test_batch')
        self.test_batch = opt.parser.get_default('test_batch')
        self.test_batch = opt.parser.get_default('test_batch')
        
        
opt = MyOpt()
dct_n = opt.dct_n
input_n = opt.input_n
output_n = opt.output_n
sample_rate = opt.sample_rate

time_autoencoder1 = TimeAutoencoder(opt.input_n + opt.output_n, dct_n)
extension = 'RAW'
name1 = 'autoencoder_' + str(opt.input_n + opt.output_n) + '_' + str(opt.dct_n) + '_' + extension + '.pt'
utils.load_model(time_autoencoder1, name1)

time_autoencoder2 = TimeAutoencoder(opt.input_n + opt.output_n, dct_n)
extension = 'SUBSAMPLED'
name2 = 'autoencoder_' + str(opt.input_n + opt.output_n) + '_' + str(opt.dct_n) + '_' + extension + '.pt'
utils.load_model(time_autoencoder2, name2)

model = nnmodel.MultipleGCN(dct_n, opt.linear_size, opt.dropout, time_autoencoder1, time_autoencoder2, opt,
                    num_stage=opt.num_stage, node_n=66)

In [37]:
best_state_dict = torch.load('/home/tim/Bureau/checkpoint/ckpt_main_3d_3D_in10_out25_dct_n_30_best.pth.tar', map_location=torch.device('cpu'))['state_dict']
GCN2_state_dict = torch.load('checkpoint/test/ckpt_main_3d_3D_in10_out25_dct_n_30_best.pth.tar', map_location=torch.device('cpu'))['state_dict']
model.load_state_dict(GCN2_state_dict)
for key_p1, key_p2 in zip([i for i in best_state_dict.keys()], [i for i in GCN2_state_dict.keys() if i.startswith('GCN1')]):
    assert(key_p1 == '.'.join(key_p2.split('.')[1:]))
    GCN2_state_dict[key_p2] = best_state_dict[key_p1]
    
# model.load_state_dict(GCN2_state_dict)

In [38]:
# Load test dataset
acts = data_utils.define_actions('all')
test_data = dict()
for act in acts:
    test_dataset = H36motion3D(path_to_data=opt.data_dir, actions=act, input_n=input_n, output_n=output_n, split=1,
                               sample_rate=sample_rate, dct_used=dct_n)
    test_data[act] = DataLoader(
        dataset=test_dataset,
        batch_size=opt.test_batch,
        shuffle=False)

# Load dummy train dataset for accessing dim_used
train_dataset = H36motion3D(path_to_data='./h3.6m/dataset', actions='all', input_n=input_n, output_n=output_n,
                          split=0, sample_rate=2, subset=True)

Reading subject 5, action walking, subaction 1
Reading subject 5, action walking, subaction 2
Reading subject 5, action eating, subaction 1
Reading subject 5, action eating, subaction 2
Reading subject 5, action smoking, subaction 1
Reading subject 5, action smoking, subaction 2
Reading subject 5, action discussion, subaction 1
Reading subject 5, action discussion, subaction 2
Reading subject 5, action directions, subaction 1
Reading subject 5, action directions, subaction 2
Reading subject 5, action greeting, subaction 1
Reading subject 5, action greeting, subaction 2
Reading subject 5, action phoning, subaction 1
Reading subject 5, action phoning, subaction 2
Reading subject 5, action posing, subaction 1
Reading subject 5, action posing, subaction 2
Reading subject 5, action purchases, subaction 1
Reading subject 5, action purchases, subaction 2
Reading subject 5, action sitting, subaction 1
Reading subject 5, action sitting, subaction 2
Reading subject 5, action sittingdown, subacti

In [39]:
from main_3d import my_evaluate
import time

In [42]:
best_alpha_vec = torch.zeros(1, 1, 35).float()

for time_index in range(34, 35):
    print("Time index {}".format(time_index))
    best_alpha, min_err = 0, 1e9
    for a in reversed(np.linspace(0.9, 1, 10)):
        start = time.time()
        tmp = 0
        for act in ['walking', 'eating', 'smoking', 'discussion']:
            err_3d = my_evaluate(test_data[act], model, train_dataset.dim_used, opt, alpha=a, j=24, dim=time_index)
            tmp += err_3d
        avg = tmp / len(['walking', 'eating', 'smoking', 'discussion'])
        print(a, avg)
        if avg < min_err:
            min_err = avg
            best_alpha = a
#         print('Time taken: {}'.format(time.time()-start))
    print('Best alpha: {}'.format(best_alpha))
#     best_alpha_vec[time_index] = best_alpha
    
    print()

Time index 34
1.0 70.5143404006958
0.9888888888888889 70.4613561630249
0.9777777777777777 70.43493270874023
0.9666666666666667 70.43456554412842
0.9555555555555556 70.45925235748291
0.9444444444444444 70.50753402709961
0.9333333333333333 70.57770919799805
0.9222222222222223 70.66821098327637
0.9111111111111111 70.77779197692871
0.9 70.90555191040039
Best alpha: 0.9666666666666667



In [46]:
[i for i in GCN2_state_dict.keys() if i.startswith('autoen')]

['autoencoder1.encoder.0.weight',
 'autoencoder1.encoder.0.bias',
 'autoencoder1.encoder.2.weight',
 'autoencoder1.encoder.2.bias',
 'autoencoder1.encoder.4.weight',
 'autoencoder1.encoder.4.bias',
 'autoencoder1.decoder.0.weight',
 'autoencoder1.decoder.0.bias',
 'autoencoder1.decoder.2.weight',
 'autoencoder1.decoder.2.bias',
 'autoencoder1.decoder.4.weight',
 'autoencoder1.decoder.4.bias',
 'autoencoder2.encoder.0.weight',
 'autoencoder2.encoder.0.bias',
 'autoencoder2.encoder.2.weight',
 'autoencoder2.encoder.2.bias',
 'autoencoder2.encoder.4.weight',
 'autoencoder2.encoder.4.bias',
 'autoencoder2.decoder.0.weight',
 'autoencoder2.decoder.0.bias',
 'autoencoder2.decoder.2.weight',
 'autoencoder2.decoder.2.bias',
 'autoencoder2.decoder.4.weight',
 'autoencoder2.decoder.4.bias']